# **TODOs**

---

> * Apply **GA** and **neuro-fuzzy** like _ANFIS_ to build a hybrid intelligence system for best rules selection and membership function creation.

**⚙️ Import neccassry modules for the app ⚙️**

In [122]:
import plotly.graph_objects as go
import numpy as np
from db import init

## **📝 DB Ops**

---

**🔰 SEE [`init`](https://github.com/wildonion/fuzzy-drugstore/blob/main/db/__init__.py) CLASS**

> * First we created a table called `drugs_info` with `id`, `name` and the `amount` of each medicine.
> * Second we inserted some fake data into the table. 
> * Finally we fetched all of them and print them nicely.

**⚠️ We limited the number of total drugs in each sample to 100 for testing purposes.**

In [2]:

FLAG_INSERT = False

db = init()
q_drugs_info = "CREATE TABLE IF NOT EXISTS drugs_info (id int, name text, current int)"
q_sales_info = "CREATE TABLE IF NOT EXISTS drugs_sales_info (durg_id int, past_month_sales int, annual_sales int)"
db.query(q_drugs_info,[])
db.query(q_sales_info,[])

if FLAG_INSERT:
    for i in range(50):
        q_di = "INSERT INTO drugs_info VALUES (?, ?, ?)"
        q_si = "INSERT INTO drugs_sales_info VALUES (?, ?, ?)"
        v_di = [i, f'cit_{i}', np.random.randint(101)]
        v_si = [i, np.random.randint(101), np.random.randint(101)]
        db.query(q_di,v_di)
        db.query(q_si,v_si)


## **📝 Membership Function Generators**

---

📚 *References*:

➱ [Membership Functions](http://www.dma.fi.upm.es/recursos/aplicaciones/logica_borrosa/web/fuzzy_inferencia/funpert_en.htm)

➱ [Gaussian](https://github.com/scikit-fuzzy/scikit-fuzzy/blob/eecf303b701e3efacdc9b9066207ef605d4facaa/skfuzzy/membership/generatemf.py#L65)

➱ [Triangular](https://github.com/scikit-fuzzy/scikit-fuzzy/blob/eecf303b701e3efacdc9b9066207ef605d4facaa/skfuzzy/membership/generatemf.py#L414)

In [3]:

def trimf(x, abc):
    assert len(abc) == 3, 'abc parameter must have exactly three elements.'
    a, b, c = np.r_[abc]
    assert a <= b and b <= c, 'abc requires the three elements a <= b <= c.'

    y = np.zeros(len(x))

    # Left side
    if a != b:
        idx = np.nonzero(np.logical_and(a < x, x < b))[0]
        y[idx] = (x[idx] - a) / float(b - a)

    # Right side
    if b != c:
        idx = np.nonzero(np.logical_and(b < x, x < c))[0]
        y[idx] = (c - x[idx]) / float(c - b)

    idx = np.nonzero(x == b)
    y[idx] = 1
    return y


def gaussmf(x, mean, sigma):
    return np.exp(-((x - mean)**2.) / (2 * sigma**2.))

## **📝 Defining Universe Range**

---

⚠️ **We limited the universe to 100.**

In [4]:
universe = np.arange(0, 100, 1)

## **📝 Antecedent - Current Drug**

---

**👉 `X` is our antecedent and the linguistic variable for the amount of current drug in our warehouse.**

> * First we picked up three linguistic values with range `[0, 36), [30, 51) and [45, 101)` from our `universe` for `low`, `medium` and `high` respectively for `X.`
> * Second we used triangular membership function to generate membership degree for each defined fuzzy set (linguistic values).
> * Finally we plotted the line of each fuzzy set using their universe and their corresponding membership degree.


In [170]:

class X: # linguistic variable
    low    = None # linguistic value for X
    medium = None
    high   = None
    name   = "current drug"
        

class _range:
    def __init__(self, lingvar, u, m=None):
        self.universe = u
        self.mf       = m
        self.lingvar  = lingvar

        

# ---------------------------------
# setting up universe for each sets
# ---------------------------------


X.low    = _range(lingvar="low", u=universe[0:36])
X.medium = _range(lingvar="medium", u=universe[30:51])
X.high   = _range(lingvar="high", u=universe[45:101])


# -----------------------------
# generating membership degrees
# -----------------------------


X.low.mf    = trimf(X.low.universe, [0, 20, 35])
X.medium.mf = trimf(X.medium.universe, [30, 40, 50])
X.high.mf   = trimf(X.high.universe, [45, 60, 75])



# -----------------------------
# plotting membership functions
# -----------------------------

fig = go.Figure()
fig.add_trace(go.Scatter(x=X.low.universe, y=X.low.mf, mode='lines', name='low'))
fig.add_trace(go.Scatter(x=X.medium.universe, y=X.medium.mf, mode='lines', name='medium'))
fig.add_trace(go.Scatter(x=X.high.universe, y=X.high.mf, mode='lines', name='high'))
fig.update_xaxes(title_text='universe')
fig.update_yaxes(title_text='degree of membership')
fig.show()


## **📝 Antecedent - Drug Sales over the Past Month**

---

**👉 `Y` is our antecedent and the linguistic variable for the amount of selling drug over the past month.**

> * First we picked up three linguistic values with range `[0, 21), [15, 36), [30, 51), [45, 66) and [60, 101)` from our `universe` for `very_few`, `few`, `medium`, `many` and `alot` respectively for `Y.`
> * Second we used triangular membership function to generate membership degree for each defined fuzzy set (linguistic values).
> * Finally we plotted the line of each fuzzy set using their universe and their corresponding membership degree.

In [171]:


class Y: # linguistic variable
    very_few = None # linguistic value
    few      = None
    medium   = None
    many     = None
    alot     = None
    name     = "sales over the past month"

        

class _range:
    def __init__(self, lingvar, u, m=None):
        self.universe = u
        self.mf       = m
        self.lingvar  = lingvar

        

# ---------------------------------
# setting up universe for each sets
# ---------------------------------


Y.very_few = _range(lingvar="very_few", u=universe[0:21])
Y.few      = _range(lingvar="few", u=universe[15:36])
Y.medium   = _range(lingvar="medium", u=universe[30:51])
Y.many     = _range(lingvar="many", u=universe[45:66])
Y.alot     = _range(lingvar="alot", u=universe[60:101])


# -----------------------------
# generating membership degrees
# -----------------------------


Y.very_few.mf = trimf(Y.very_few.universe, [0, 10, 20])
Y.few.mf      = trimf(Y.few.universe, [15, 25, 35])
Y.medium.mf   = trimf(Y.medium.universe, [30, 40, 50])
Y.many.mf     = trimf(Y.many.universe, [45, 55, 65])
Y.alot.mf     = trimf(Y.alot.universe, [60, 65, 75])


# -----------------------------
# plotting membership functions
# -----------------------------

fig = go.Figure()
fig.add_trace(go.Scatter(x=Y.very_few.universe, y=Y.very_few.mf, mode='lines', name='very few'))
fig.add_trace(go.Scatter(x=Y.few.universe, y=Y.few.mf, mode='lines', name='few'))
fig.add_trace(go.Scatter(x=Y.medium.universe, y=Y.medium.mf, mode='lines', name='medium'))
fig.add_trace(go.Scatter(x=Y.many.universe, y=Y.many.mf, mode='lines', name='many'))
fig.add_trace(go.Scatter(x=Y.alot.universe, y=Y.alot.mf, mode='lines', name='alot'))
fig.update_xaxes(title_text='universe')
fig.update_yaxes(title_text='degree of membership')
fig.show()

## **📝 Antecedent - Drug Sales over the Past Years**

---

**👉 `Z` is our antecedent and the linguistic variable for the amount of selling drug over the past years in current season.**

> * First we picked up three linguistic values with range `[0, 21), [15, 36), [30, 51), [45, 66) and [60, 101)` from our `universe` for `very_few`, `few`, `medium`, `many` and `alot` respectively for `Z.`
> * Second we used triangular membership function to generate membership degree for each defined fuzzy set (linguistic values).
> * Finally we plotted the line of each fuzzy set using their universe and their corresponding membership degree.

In [172]:

class Z: # linguistic variable
    very_few = None # linguistic value
    few      = None
    medium   = None
    many     = None
    alot     = None
    name     = "sales over the past years"

        

class _range:
    def __init__(self, lingvar, u, m=None):
        self.universe = u
        self.mf       = m
        self.lingvar  = lingvar

        


# ---------------------------------
# setting up universe for each sets
# ---------------------------------


Z.very_few = _range(lingvar="very_few", u=universe[0:21])
Z.few      = _range(lingvar="few", u=universe[15:36])
Z.medium   = _range(lingvar="medium", u=universe[30:51])
Z.many     = _range(lingvar="many", u=universe[45:66])
Z.alot     = _range(lingvar="alot", u=universe[60:101])


# -----------------------------
# generating membership degrees
# -----------------------------


Z.very_few.mf = trimf(Z.very_few.universe, [0, 10, 20])
Z.few.mf      = trimf(Z.few.universe, [15, 25, 35])
Z.medium.mf   = trimf(Z.medium.universe, [30, 40, 50])
Z.many.mf     = trimf(Z.many.universe, [45, 55, 65])
Z.alot.mf     = trimf(Z.alot.universe, [60, 65, 75])



# -----------------------------
# plotting membership functions
# -----------------------------

fig = go.Figure()
fig.add_trace(go.Scatter(x=Z.very_few.universe, y=Z.very_few.mf, mode='lines', name='very few'))
fig.add_trace(go.Scatter(x=Z.few.universe, y=Z.few.mf, mode='lines', name='few'))
fig.add_trace(go.Scatter(x=Z.medium.universe, y=Z.medium.mf, mode='lines', name='medium'))
fig.add_trace(go.Scatter(x=Z.many.universe, y=Z.many.mf, mode='lines', name='many'))
fig.add_trace(go.Scatter(x=Z.alot.universe, y=Z.alot.mf, mode='lines', name='alot'))
fig.update_xaxes(title_text='universe')
fig.update_yaxes(title_text='degree of membership')
fig.show()


## **📝 Consequent - Amount of Order**

---

**👉 `W` is our consequent and the linguistic variable for the amount of ordering drug for our warehouse.**

> * First we picked up three linguistic values with range `[0, 36), [30, 51) and [45, 101)` from our `universe` for `low`, `medium` and `high` respectively for `W.`
> * Second we used triangular membership function to generate membership degree for each defined fuzzy set (linguistic values).
> * Finally we plotted the line of each fuzzy set using their universe and their corresponding membership degree.

In [173]:

class W: # linguistic variable
    low    = None # linguistic value
    medium = None
    high   = None
    name   = "amount of order" 


class _range:
    def __init__(self, lingvar, u, m=None):
        self.universe = u
        self.mf       = m
        self.lingvar  = lingvar



# ---------------------------------
# setting up universe for each sets
# ---------------------------------


W.low    = _range(lingvar="low", u=universe[0:36])
W.medium = _range(lingvar="medium", u=universe[30:51])
W.high   = _range(lingvar="high", u=universe[45:101])


# -----------------------------
# generating membership degrees
# -----------------------------


W.low.mf      = trimf(W.low.universe, [0, 20, 35])
W.medium.mf   = trimf(W.medium.universe, [30, 40, 50])
W.high.mf     = trimf(W.high.universe, [45, 60, 75])


# -----------------------------
# plotting membership functions
# -----------------------------

fig = go.Figure()
fig.add_trace(go.Scatter(x=W.low.universe, y=W.low.mf, mode='lines', name='low'))
fig.add_trace(go.Scatter(x=W.medium.universe, y=W.medium.mf, mode='lines', name='medium'))
fig.add_trace(go.Scatter(x=W.high.universe, y=W.high.mf, mode='lines', name='high'))
fig.update_xaxes(title_text='universe')
fig.update_yaxes(title_text='degree of membership')
fig.show()




## **📝 Rule Class - Fuzzy Rules Object**

---

> * We defined a `Rule` class for our fuzzy rules with `left_c`, `right_c`, `operator` and `_then_` attributes.
> * In order to build the rules we have to build an object from this class for each rule and fill its attributes with correct values.

In [174]:

class Rule:
    def __init__(self, left_c=None, operator="and", right_c=None, _then=None):
        self.left_c   = left_c
        self.operator = operator
        self.right_c  = right_c
        self._then    = _then # output for amount of order - consequent W
        


## **📝 Rule Base 1 - Drug Sales over the Past Month**

---

**👉 We built 15 objects of our `Rule` class based on selling drug over the past month rules table.**

In [175]:

month_rules = []

month_rules.append(Rule(left_c=Y.alot, operator="and", right_c=X.low,    _then=W.high))
month_rules.append(Rule(left_c=Y.alot, operator="and", right_c=X.medium, _then=W.high))
month_rules.append(Rule(left_c=Y.alot, operator="and", right_c=X.high,   _then=W.high))

month_rules.append(Rule(left_c=Y.many, operator="and", right_c=X.low,    _then=W.high))
month_rules.append(Rule(left_c=Y.many, operator="and", right_c=X.medium, _then=W.high))
month_rules.append(Rule(left_c=Y.many, operator="and", right_c=X.high,   _then=W.medium))

month_rules.append(Rule(left_c=Y.medium, operator="and", right_c=X.low,    _then=W.high))
month_rules.append(Rule(left_c=Y.medium, operator="and", right_c=X.medium, _then=W.medium))
month_rules.append(Rule(left_c=Y.medium, operator="and", right_c=X.high,   _then=W.low))

month_rules.append(Rule(left_c=Y.few, operator="and", right_c=X.low,    _then=W.medium))
month_rules.append(Rule(left_c=Y.few, operator="and", right_c=X.medium, _then=W.low))
month_rules.append(Rule(left_c=Y.few, operator="and", right_c=X.high,   _then=W.low))

month_rules.append(Rule(left_c=Y.very_few, operator="and", right_c=X.low,    _then=W.low))
month_rules.append(Rule(left_c=Y.very_few, operator="and", right_c=X.medium, _then=W.low))
month_rules.append(Rule(left_c=Y.very_few, operator="and", right_c=X.high,   _then=W.low))


## **📝 Rule Base 2 - Drug Sales over the Past Years**

---

**👉 We built 15 objects of our `Rule` class based on selling drug over the past years rules table.**

In [176]:
 
years_rules = []

years_rules.append(Rule(left_c=Z.alot, operator="and", right_c=X.low,    _then=W.high))
years_rules.append(Rule(left_c=Z.alot, operator="and", right_c=X.medium, _then=W.high))
years_rules.append(Rule(left_c=Z.alot, operator="and", right_c=X.high,   _then=W.high))

years_rules.append(Rule(left_c=Z.many, operator="and", right_c=X.low,    _then=W.high))
years_rules.append(Rule(left_c=Z.many, operator="and", right_c=X.medium, _then=W.high))
years_rules.append(Rule(left_c=Z.many, operator="and", right_c=X.high,   _then=W.medium))

years_rules.append(Rule(left_c=Z.medium, operator="and", right_c=X.low,    _then=W.high))
years_rules.append(Rule(left_c=Z.medium, operator="and", right_c=X.medium, _then=W.medium))
years_rules.append(Rule(left_c=Z.medium, operator="and", right_c=X.high,   _then=W.low))

years_rules.append(Rule(left_c=Z.few, operator="and", right_c=X.low,    _then=W.medium))
years_rules.append(Rule(left_c=Z.few, operator="and", right_c=X.medium, _then=W.low))
years_rules.append(Rule(left_c=Z.few, operator="and", right_c=X.high,   _then=W.low))

years_rules.append(Rule(left_c=Z.very_few, operator="and", right_c=X.low,    _then=W.low))
years_rules.append(Rule(left_c=Z.very_few, operator="and", right_c=X.medium, _then=W.low))
years_rules.append(Rule(left_c=Z.very_few, operator="and", right_c=X.high,   _then=W.low))


## **📝 Defuzzification Process - Computing Output for Database Samples Based on a Specific Rule**

--- 

> * We used the `COG` (Center of Gravity) algorithm for defuzzification process to find the drug order risk and the amount of it for each linguistic variable (fuzzy sets) over one of the specified rules (month or years).
> * For building left condition rules we have to check that what universe of linguistic variable `Y` value contains the number of sold drug sample over the past month.
> * For building left condition rules we have to check that what universe of linguistic variable `Z` value contains the number of sold drug sample over the years.
> * For building left right rules we have to check that what universe of linguistic variable `X` value contains the amount of current drug sample.
> * Finally we iterated through all specified rules to generate the amount of order and the risk of that amount for a specific sample.

In [177]:

def compute(drug, verbose=True):
    

    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    #           DEFINING NECESSARY VARIABLES BASED ON rule_base
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    rule_base                    = "month" if "Y" in list(drug.keys()) else "years"
    outputs                      = {}
    cog                          = []
    defuzzification_list         = []
    overlapping_right_conditions = [] # Maximum length is two because at most two triangular diagrams will overlap.
    overlapping_left_conditions  = [] # Maximum length is two because at most two triangular diagrams will overlap.
    rules_to_iterate             = years_rules if rule_base == "years" else month_rules
    defuzz_print_msg             = "YEARS IN CURRENT SEASON" if rule_base == "years" else "MONTH"
    rule_base_amount             = drug["Y"] if rule_base == "month" else drug["Z"]
    
    
    
    
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    #       DEFUZZIFICATION PROCESS FOR rule_base USING COG METHOD
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    for i in range(len(rules_to_iterate)):
        w_mf  = rules_to_iterate[i]._then.mf
        w_uni = rules_to_iterate[i]._then.universe
        umf = [[k for k,v in dict(zip(w_uni, w_mf)).items() if v == w_mf[i]] for i in range(len(w_mf))]
        umf = {w_mf[i] : umf[i] for i in range(len(w_mf))}
        sigma_uXm = 0
        sigma_mf  = 0
        for m, uni in umf.items():
            sigma_uXm += m*sum(uni)
            sigma_mf  += m*len(uni)
        cog.append(sigma_uXm/sigma_mf)
    if verbose:
        print(f"✅ DEFUZZIFICATION RESLUTS FOR DRUG SALES OVER THE PAST {defuzz_print_msg} RULES")
    for j in range(len(cog)):
        defuzzification_list.append({"order amount": rules_to_iterate[j]._then.lingvar, "risk": f"{cog[j]:0.2f}"})
        if verbose:
            print(f"\t🔴 Order Amount ::: {rules_to_iterate[j]._then.lingvar} \n\t⚠️ Risk To Take ::: {cog[j]:0.2f}% ")
            print("\t≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛")
    
    
    
    
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    # BUILDING RIGHT RULES FOR OUR SAMPLE BASED 
    # ON ITS DRUG INFOS TO FIND THE ORDER AMOUNT AND RISK
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    if drug["X"] in X.low.universe:
        overlapping_right_conditions.append(X.low)
    if drug["X"] in X.medium.universe:
        overlapping_right_conditions.append(X.medium)
    if drug["X"] in X.high.universe:
        overlapping_right_conditions.append(X.high)
    
    
    
    
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    # BUILDING LEFT RULES BASED ON rule_base FOR OUR SAMPLE BASED 
    # ON ITS DRUG INFOS TO FIND THE ORDER AMOUNT AND RISK
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    if rule_base == "month":
        if drug["Y"] in Y.very_few.universe:
            overlapping_left_conditions.append(Y.very_few)
        if drug["Y"] in Y.few.universe:
            overlapping_left_conditions.append(Y.few)
        if drug["Y"] in Y.medium.universe:
            overlapping_left_conditions.append(Y.medium)
        if drug["Y"] in Y.many.universe:
            overlapping_left_conditions.append(Y.many)
        if drug["Y"] in Y.alot.universe:
            overlapping_left_conditions.append(Y.alot)
    elif rule_base == "years":
        if drug["Z"] in Z.very_few.universe:
            overlapping_left_conditions.append(Z.very_few)
        if drug["Z"] in Z.few.universe:
            overlapping_left_conditions.append(Z.few)
        if drug["Z"] in Z.medium.universe:
            overlapping_left_conditions.append(Z.medium)
        if drug["Z"] in Z.many.universe:
            overlapping_left_conditions.append(Z.many)
        if drug["Z"] in Z.alot.universe:
            overlapping_left_conditions.append(Z.alot)
    else:
        raise ValueError("❌ unknown rule base!")



    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    # ITERATE THROUGH ALL RULES TO FIND THE SAMPLE RULES AND 
    # SAVE THEM WITH THEIR AMOUNT ORDER AND RISKS VALUES IN OUTPUTS DICT
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    for i in range(len(rules_to_iterate)):

        if len(overlapping_left_conditions) == 2 and len(overlapping_right_conditions) == 2:
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]
            if overlapping_left_conditions[1] == rules_to_iterate[i].left_c and overlapping_right_conditions[1] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[1] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]
            if overlapping_left_conditions[1] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]

        elif len(overlapping_left_conditions) == 2 and len(overlapping_right_conditions) == 1:
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]
            if overlapping_left_conditions[1] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]

        elif len(overlapping_left_conditions) == 1 and len(overlapping_right_conditions) == 2:
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[1] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]

        elif len(overlapping_left_conditions) == 1 and len(overlapping_right_conditions) == 1:
            if overlapping_left_conditions[0] == rules_to_iterate[i].left_c and overlapping_right_conditions[0] == rules_to_iterate[i].right_c: # comparing memory address
                for rule_result in defuzzification_list:
                    if rule_result["order amount"] == rules_to_iterate[i]._then.lingvar:
                        outputs[rules_to_iterate[i]._then.lingvar] = rule_result["risk"]


                        

    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    #                        PRINTING THE OUTPUTS INFOS
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    lingvars              = list(outputs.keys())
    risks                 = list(outputs.values())
    amounts_to_be_ordered = f"{lingvars[0]} & {lingvars[1]}" if len(lingvars) == 2 else lingvars[0]
    risks_to_be_taken     = f"{risks[0]}% & {risks[1]}%" if len(risks) == 2 else f"{risks[0]}%"
    drug_name             = drug["name"] if "name" in list(drug.keys()) else "--NONE--"
    vertical_lines        = [dict(type='line', yref='paper', y0= 0, y1= 1, xref='x', x0=int(float(risks[0])), x1=int(float(risks[0]))),
                             dict(type='line', yref='paper', y0= 0, y1= 1, xref='x', x0=int(float(risks[1])), x1=int(float(risks[1])))] if len(risks) == 2 else \
                            [dict(type='line', yref='paper', y0= 0, y1= 1, xref='x', x0=int(float(risks[0])), x1=int(float(risks[0])))]
    if verbose:
        print()
        print("≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣")
        print(f"📌 DRUG NAME                   ▶ ", drug_name)
        print(f"📌 CURRENT AMOUNT              ▶ ", drug["X"])
        print(f"📌 PAST {rule_base.upper()} SALES AMOUNT     ▶ ", rule_base_amount)
        print(f"📌 THE AMOUNT(s) TO BE ORDERED ▶ ", amounts_to_be_ordered)
        print(f"📌 THE RISK(s) TO BE TAKEN     ▶ ", risks_to_be_taken)
        print("≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣")

    
    
    
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    #                         PLOTTING THE OUTPUTS
    # ≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣≣
    low_fill, low_name, medium_fill, medium_name, high_fill, high_name = None, 'low', None, 'medium', None, 'high'
    if lingvars[0] == "low": low_fill = "toself"; low_name = "<b>low</b>"
    elif lingvars[0] == "medium": medium_fill = "toself"; medium_name = "<b>medium</b>"
    else: high_fill = "toself"; high_name = "<b>high</b>"
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=W.low.universe, y=W.low.mf, mode='lines', name=low_name, fill=low_fill))
    fig.add_trace(go.Scatter(x=W.medium.universe, y=W.medium.mf, mode='lines', name=medium_name, fill=medium_fill))
    fig.add_trace(go.Scatter(x=W.high.universe, y=W.high.mf, mode='lines', name=high_name, fill=high_fill))
    fig.update_xaxes(title_text='universe')
    fig.update_yaxes(title_text='degree of membership')
    fig.update_layout(title=f'<b>Amount of Order - Based on Past【{rule_base.upper()}】Sales</b>')
    fig.update_layout(shapes=vertical_lines)

    return outputs, fig

## **📝 Testing the System**

---

**💊 Taking a Sample From Dataset**

> * `drug[X]` is the current amount of the sample.
> * `drug[Y]` is the number of past month sales of the sample. 
> * `drug[Z]` is the number of past years sales of the sample.
> * **Line 16**: change `Z` to `Y` if you want to compute the output (amount of order - `W`) based on past years and run the cell again.

In [196]:

q_di_fetch  = "select * from drugs_info"
drugs = db.query(q_di_fetch,[])
di = list(drugs.fetchall())

q_si_fetch = "select * from drugs_sales_info"
sales_info = db.query(q_si_fetch,[])
si = list(sales_info.fetchall())

drug_info_sales_info_pair = dict(zip(di, si))
total_drugs_info = [{"name": di[1], "X": di[2], "Y": si[1], "Z": si[2]} for di, si in drug_info_sales_info_pair.items()]


drug = total_drugs_info[40]
print(drug)
drug = {k: v for k, v in drug.items() if k != 'Z'}
print(drug)

{'name': 'cit_40', 'X': 30, 'Y': 59, 'Z': 36}
{'name': 'cit_40', 'X': 30, 'Y': 59}


**📢 `outputs` is a dictionary of order amount and the risk to be taken.**

**📢 `fig` is a plotly figure object of the output. You can call its `show()` method to plot the output.**

---

In [197]:
outputs, fig = compute(drug=drug, verbose=True)
fig.show()

✅ DEFUZZIFICATION RESLUTS FOR DRUG SALES OVER THE PAST MONTH RULES
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: medium 
	⚠️ Risk To Take ::: 40.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: high 
	⚠️ Risk To Take ::: 60.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: medium 
	⚠️ Risk To Take ::: 40.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: low 
	⚠️ Risk To Take ::: 18.33% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: medium 
	⚠️ Risk To Take ::: 40.00% 
	≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛≛
	🔴 Order Amount ::: low 
	⚠️ Risk To Take ::: 18.33% 
	≛≛≛

## **📊 3D Plots - View the System**

---

In [199]:

x, y = np.meshgrid(universe, universe)
w = np.zeros_like(x)
n = universe.shape[0]

# loop through the system n*n times to collect the control surface
for i in range(n):
    for j in range(n):
        current          = x[i, j]
        past_month_sales = y[i, j]
        outputs, _ = compute(drug={"X": current, "Y": month}, verbose=False)
        w[i, j] = float(list(outputs.values())[0]) # collect the risk of the output or order amount

        
fig = go.Figure(data=go.Surface(x=x, y=y, z=w))
fig.update_layout(layout, width=700, margin=dict(r=20, l=10, b=10, t=10))
fig.update_scenes(xaxis_title_text=X.name, yaxis_title_text=Y.name, zaxis_title_text=W.name)
fig.show()